# ML HW3


In [ ]:
import os

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from scipy.stats import multivariate_normal
from sklearn import svm

## 1. Support Vector Machine


In [ ]:
data = pd.read_csv('./x_train.csv', header=None) / 255.0
data = np.array(data)
data = data - np.mean(data, axis=0)

label = pd.read_csv('./t_train.csv', header=None)
label = np.array(label)


### Principal Component Analysis (PCA)


In [ ]:
class PCA:
    eigenvalues: np.ndarray
    eigenvectors: np.ndarray

    def __init__(self, n_components) -> None:
        self.n_components = n_components

    def fit(self, x):
        """
        Args:
            x: (n_samples, n_features)
        Returns:
            self
        """
        u, s, vh = np.linalg.svd(x, full_matrices=False)
        self.eigenvalues = np.square(s) / (x.shape[0] - 1)
        self.eigenvectors = vh.T[:, :self.n_components]
        return self

    def transform(self, x):
        """
        Args:
            x: (n_samples, n_features)
        Returns:
            (n_samples, n_components)
        """
        return x @ self.eigenvectors

    def fit_transform(self, x):
        return self.fit(x).transform(x)

    def components(self):
        """
        Returns:
            (n_components, n_features)
        """
        return self.eigenvectors.T

    def explained_variance(self):
        return self.eigenvalues[:self.n_components]

    def explained_variance_ratio(self):
        return self.eigenvalues[:self.n_components] / np.sum(self.eigenvalues)

In [ ]:
pca = PCA(n_components=2)
pca_data = pca.fit_transform(data)
print(pca_data.shape)

### Decision Approaches

1. one-vs-one

   Choose two classes each time, and train a binary classifier for each pair of classes. Then, for a new sample, we can use the binary classifiers to predict the class of the sample. Finally, we can choose the class that has the most votes.

2. one-vs-rest

   Choose one class as the positive class, and the rest as the negative class. Then, we can train a binary classifier for each class. Finally, we can choose the class that has the most votes.

In this case, both approaches will train three binary classifiers. Here. I will use **one-vs-one** approach because it use only two classes of data each time, and it is more efficient than one-vs-rest approach.


### Support Vector Machine (SVM)


In [ ]:
from typing import Literal


class Kernel:

    def __init__(self, mode: Literal['Linear', 'Polynomial']) -> None:
        if mode not in ['Linear', 'Polynomial']:
            raise ValueError(f'Unknown mode: {mode}, should be `Linear` or `Polynomial`')

        self.mode = mode

    def phi(self, x: np.ndarray):
        if self.mode == 'Linear':
            return x
        else:  # self.mode == 'Polynomial':
            return np.array([
                np.square(x[..., 0]),
                np.sqrt(2) * x[..., 0] * x[..., 1],
                np.square(x[..., 1]),
            ]).T

    def compute_kernel(self, x1, x2):
        return self.phi(x1) @ self.phi(x2).T


In [ ]:
class SVM:
    coef: np.ndarray
    sv_idx: np.ndarray
    weight: np.ndarray
    bias: np.floating

    def __init__(self, pos_cls, neg_cls, kernel: Kernel, C=1.0) -> None:
        self.kernel = kernel
        self.C = C
        self.pos_cls = pos_cls
        self.neg_cls = neg_cls

    def compute_weight_and_bias(self, alpha, x, y):
        weight = (alpha * y).T @ self.kernel.phi(x)

        m_idx = np.bitwise_and(self.C > alpha, alpha > 0).reshape(-1)
        s_idx = (alpha != 0).reshape(-1)
        kernels = self.kernel.compute_kernel(x[m_idx], x[s_idx])
        bias = np.mean(y[m_idx] - kernels @ (alpha * y)[s_idx]) if len(m_idx) > 0 else 0.0

        return weight, bias

    def fit(self, x, y):
        y = np.where(y == self.pos_cls, 1, -1)

        from sklearn.svm import SVC
        if self.kernel.mode == 'Linear':
            clf = SVC(kernel='linear', C=self.C, decision_function_shape='ovo')
        else:  # self.kernel.mode == 'Linear'
            clf = SVC(kernel='poly', C=self.C, degree=2, decision_function_shape='ovo')
        clf.fit(x, y)

        self.coef = np.abs(clf.dual_coef_.T)
        self.sv_idx = clf.support_
        alpha = np.zeros_like(y)
        alpha[self.sv_idx] = self.coef
        self.weight, self.bias = self.compute_weight_and_bias(alpha, x, y)
        return self

    def predict(self, x):
        pred = np.sign(self.kernel.phi(x) @ self.weight.T + self.bias)
        return np.where(pred == 1, self.pos_cls, self.neg_cls)


In [ ]:
def vote(y):
    classes = np.arange(3)
    counts = np.empty((y.shape[0], 3))
    for cls in classes:
        counts[:, cls] = np.sum(y == cls, axis=-1)

    return np.argmax(counts, axis=-1)

In [ ]:
linear_kernel = Kernel(mode='Linear')
C = 1.0
linear_svm = []

for c1, c2 in [[0, 1], [0, 2], [1, 2]]:
    idx = np.logical_or(label == c1, label == c2).reshape(-1)
    x = pca_data[idx]
    y = label[idx]
    linear_svm.append(SVM(c1, c2, linear_kernel, C=C).fit(x, y))


In [ ]:
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy


def plot(x, y, y_pred, sv_idx, xx, yy, title):
    c1_idx = np.where(y == 0)
    c2_idx = np.where(y == 1)
    c3_idx = np.where(y == 2)
    plt.figure(figsize=(10, 10))
    plt.title(title)
    plt.scatter(x[sv_idx, 0],
                x[sv_idx, 1],
                facecolors='none',
                edgecolors='k',
                linewidths=1.25,
                label='Support Vector')
    plt.scatter(x[c1_idx, 0], x[c1_idx, 1], c='r', marker='x', label='T-shirt/top')
    plt.scatter(x[c2_idx, 0], x[c2_idx, 1], c='b', marker='x', label='Trouser')
    plt.scatter(x[c3_idx, 0], x[c3_idx, 1], c='g', marker='x', label='Sandal')
    plt.contourf(xx, yy, y_pred, alpha=0.3, cmap=plt.cm.brg)
    plt.legend()
    plt.show()

In [ ]:
xx, yy = make_meshgrid(pca_data[:, 0], pca_data[:, 1])
print(xx.shape, yy.shape)

# plot(pca_data, label, y_pred, sv_idx, xx, yy, 'Linear SVM')

## 2. Gaussian Mixture Model


In [ ]:
with Image.open("./hw3.jpg") as img:
    image = img.copy()